In [63]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

d = ctrl.Antecedent(np.arange(0, 101, 1), 'zadany_zasieg')
alpha = ctrl.Antecedent(np.arange(0, 91, 1), 'kat_rzutu')
k = ctrl.Antecedent(np.arange(0, 3.1, 0.1), 'wspolczynnik_oporu')
v0 = ctrl.Consequent(np.arange(0, 101, 1), 'predkosc_poczatkowa')

d['krotki'] = fuzz.trimf(d.universe, [0, 0, 50])
d['sredni'] = fuzz.trimf(d.universe, [25, 50, 75])
d['dlugi'] = fuzz.trimf(d.universe, [50, 100, 100])

alpha['male'] = fuzz.trimf(alpha.universe, [0, 0, 45])
alpha['sredni'] = fuzz.trimf(alpha.universe, [30, 45, 60])
alpha['duzy'] = fuzz.trimf(alpha.universe, [45, 90, 90])

k['maly'] = fuzz.trimf(k.universe, [0, 0, 1])
k['sredni'] = fuzz.trimf(k.universe, [0.5, 1.5, 2.5])
k['duzy'] = fuzz.trimf(k.universe, [2.5, 5, 5])

v0['mala'] = fuzz.trimf(v0.universe, [0, 0, 50])
v0['srednia'] = fuzz.trimf(v0.universe, [25, 50, 75])
v0['duza'] = fuzz.trimf(v0.universe, [50, 100, 100])

# d - zasieg rzutu, alpha - kat rzutu, k - wsp oporu powietrza
rules = [
    ctrl.Rule(d['krotki'] & alpha['male'] & k['maly'], v0['srednia']),
    ctrl.Rule(d['sredni'] & alpha['sredni'] & k['sredni'], v0['srednia']),
    ctrl.Rule(d['dlugi'] & alpha['duzy'] & k['duzy'], v0['duza']),
    ctrl.Rule(d['krotki'] & alpha['sredni'] & k['maly'], v0['mala']),
    ctrl.Rule(d['sredni'] & alpha['male'] & k['maly'], v0['srednia']),
    ctrl.Rule(d['dlugi'] & alpha['duzy'] & k['maly'], v0['srednia'])
]

model = ctrl.ControlSystem(rules)
v0est = ctrl.ControlSystemSimulation(model)

def szacuj_predkosc_poczatkowa(zadany_zasieg_wartosc, kat_rzutu_wartosc, wspolczynnik_oporu_wartosc):
    v0est.input['zadany_zasieg'] = zadany_zasieg_wartosc
    v0est.input['kat_rzutu'] = kat_rzutu_wartosc
    v0est.input['wspolczynnik_oporu'] = wspolczynnik_oporu_wartosc
    v0est.compute()
    return v0est.output['predkosc_poczatkowa']

zadany_zasieg_wartosc = 8.5
kat_rzutu_wartosc = 45
wspolczynnik_oporu_wartosc = 0.2

predkosc_pocz = szacuj_predkosc_poczatkowa(zadany_zasieg_wartosc, kat_rzutu_wartosc, wspolczynnik_oporu_wartosc)
print("Szacowana prędkość początkowa dla zasięgu rzutu", zadany_zasieg_wartosc, "to:", predkosc_pocz)


Szacowana prędkość początkowa dla zasięgu rzutu 8.5 to: 17.222222222222218


In [62]:
import math

def zasieg_rzutu(V0, alpha, k):
    g = 9.81
    xmax = (V0**2 * math.sin(math.radians(2 * alpha))) / (g + k * V0)
    return xmax

# Metoda zlotego podzialu
def znajdz_poczatkowa_predkosc(d, alfa, k, epsilon=1):
    a = 0.1
    b = 100
    
    while True:
        V0 = (a + b) / 2
        xmax = zasieg_rzutu(V0, alfa, k)

        if abs(xmax - d) < epsilon:
            return V0
        elif xmax < d:
            a = V0
        else:
            b = V0

zasieg = 8.5
kat = 45
opor_powietrza = 0.2

V0 = znajdz_poczatkowa_predkosc(zasieg, kat, opor_powietrza)
print("Prędkość początkowa rzutu ukośnego:", V0, "m/s")


Prędkość początkowa rzutu ukośnego: 9.465625 m/s
